In [1]:
import sys
import threading
import time
import zmq
import ceo
import numpy as np
from collections import OrderedDict
import os
import shelve
import traceback
import scipy.linalg as LA
import pickle
import zlib
import logging
import copy
from numpy.linalg import norm

from ruamel.yaml import YAML
yaml=YAML(typ='safe')

In [2]:
path_to_config_dir = './dos/trivial_AcO'

In [3]:
logger = logging.getLogger('Teste')
logger.setLevel(logging.INFO)

DOS_REPO = path_to_config_dir
cfg_file = os.path.join(path_to_config_dir,'dos.yaml')
logger.info('Reading config from %s',cfg_file)
with open(cfg_file) as f:
    cfg = yaml.load(f)

In [4]:
agent = None
N_SAMPLE = int(cfg['simulation']['sampling frequency']*
               cfg['simulation']['duration'])

In [5]:
from dos.dos import Logs

In [6]:
__k_step = 0
pushed = False
initialized = False
tau = 1/cfg['simulation']['sampling frequency']
logs = Logs(tau,logs_repo=DOS_REPO)

In [7]:
from dos.driver import Server, Client, Atmosphere
verbose=logging.INFO

In [8]:
cfg['drivers'].keys()

dict_keys(['M1', 'M2', 'wfs48', 'AcO_controller', 'science'])

In [9]:
drivers = {}
for d,v in cfg['drivers'].items():
    prm_file = os.path.join(path_to_config_dir,d)
    if os.path.isfile(prm_file+'.yaml') or os.path.isfile(prm_file+'.pickle'):
        logger.info('New driver: %s',d)
        if 'server' in v and v['server'] is False:
            drivers[d] = Client(tau,d,
                                            logs,
                                            verbose=verbose,**v)
        elif d=='atmosphere':
            drivers[d] = Atmosphere(tau,d,agent,
                                                    verbose=verbose)
        else:
            print(v)
            drivers[d] = Server(tau,d,
                                            logs,
                                            agent,
                                            verbose=verbose,**v)
    else:
        logger.warning('%s is missing!',prm_file)

INFO:Teste:New driver: M1
INFO:M1:New input: Txyz
INFO:M1:New input: Rxyz
INFO:M1:New input: modes
INFO:M1:No inputs!
INFO:Teste:New driver: M2
INFO:M2:New input: Txyz
INFO:M2:New input: Rxyz
INFO:M2:No inputs!
INFO:Teste:New driver: wfs48
INFO:wfs48:No inputs!
INFO:wfs48:New output: M12_RBM
INFO:wfs48:Output logged in!
INFO:Teste:New driver: AcO_controller
INFO:AcO_controller:New input: x_hat
INFO:AcO_controller:New output: M1 Txyz
INFO:AcO_controller:Output logged in!
INFO:AcO_controller:New output: M1 Rxyz
INFO:AcO_controller:Output logged in!
INFO:AcO_controller:New output: M2 Txyz
INFO:AcO_controller:Output logged in!
INFO:AcO_controller:New output: M2 Rxyz
INFO:AcO_controller:Output logged in!
INFO:AcO_controller:New output: M1 BM
INFO:AcO_controller:Output logged in!
INFO:Teste:New driver: science
INFO:science:No inputs!
INFO:science:New output: wfe_rms
INFO:science:Output logged in!
INFO:science:New output: segment_wfe_rms
INFO:science:Output logged in!
INFO:science:New output:

{'delay': 0, 'sampling_rate': 1, 'inputs': {'Txyz': {'size': [7, 3]}, 'Rxyz': {'size': [7, 3]}, 'modes': {'size': [7, 27]}}}
{'delay': 0, 'sampling_rate': 1, 'inputs': {'Txyz': {'size': [7, 3]}, 'Rxyz': {'size': [7, 3]}}}
{'delay': 0, 'sampling_rate': 1, 'outputs': {'M12_RBM': {'sampling_rate': 1, 'size': 273, 'logs': {'decimation': 1}}}}
{'outputs': {'wfe_rms': {'size': 1, 'logs': {'decimation': 1}}, 'segment_wfe_rms': {'size': 7, 'logs': {'decimation': 1}}, 'PSSn': {'size': 1, 'logs': {'decimation': 1}}, 'tiptilt': {'size': 2, 'logs': {'decimation': 1}}, 'segment_piston': {'size': 7, 'logs': {'decimation': 1}}, 'segment_tiptilt': {'size': 14, 'logs': {'decimation': 1}}}}


In [15]:
 for k_d in drivers:
    d = drivers[k_d]
    for k_i in d.inputs:
        d.inputs[k_i].tie(drivers)
    for k_o in d.outputs:
        d.outputs[k_o].tie(drivers)
for k_d in drivers:
    d = drivers[k_d]
    device = os.path.join(path_to_config_dir,k_d)
    try:
        with open(device+'.yaml') as f:
            prm = yaml.load(f)
    except:
        with open(device+'.pickle','rb') as f:
            prm = pickle.load(f)
    #print(prm)
    if 'mirror' in prm:
        print(k_d)
    #print('\n\n')
    d.associate(prm)

INFO:x_hat:Linked to M12_RBM from wfs48
INFO:M1 Txyz:Linked to Txyz from M1
INFO:M1 Rxyz:Linked to Rxyz from M1
INFO:M2 Txyz:Linked to Txyz from M2
INFO:M2 Rxyz:Linked to Rxyz from M2
INFO:M1 BM:Linked to modes from M1


M1
M2


In [35]:
drivers['M1'].inputs

{'Txyz': <dos.driver.Input at 0x7f67f0173b50>,
 'Rxyz': <dos.driver.Input at 0x7f67f7f21410>,
 'modes': <dos.driver.Input at 0x7f67f016fa10>}

In [ ]:
self.__start = map(lambda x: x.start(), self.drivers.values())
self.__init = map(lambda x: x.init(), self.drivers.values())
self.step = self.stepping()
self.__terminate = map(lambda x: x.terminate(), self.drivers.values())
self.logger.info('Simulation setup for a duration of {0}s @ {1}Hz ({2} steps)!'.format(
    self.cfg['simulation']['duration'],
    self.cfg['simulation']['sampling frequency'],
    self.N_SAMPLE))
